In [1]:
import cirq
import qibo
qibo.set_backend("numpy")

import tequila as tq
import numpy as np
import copy
import sys
import time

sys.path.append('../../../../Gates_Lab_Suite')
sys.path.append('../../utils')

from Auto_Algorithm import *
from Core_Definition import *
from Visualization import *

from hamiltonian_reduction import (
    reduce_measurements_naive
)

from circuit_conversion import (
    cirq_to_gates,
    cirq_to_qibo
)

from simulation_tools import (
    get_energy_from_data,
    simulate_data_cirq,
    simulate_data_qibo,
    get_measurement_dict_from_H,
    get_energy_from_expectation_values,
    get_energy_from_data,
    convert_tq_H_to_cirq_H,
    convert_tq_H_to_dict_H,
    convert_population_data_to_expectation_values)

[Qibo 0.2.18|INFO|2025-07-16 15:37:38]: Using numpy backend on /CPU:0
/home/liamj/Molecular Simulation/chemical-graph/tequila-beta/lib/python3.12/site-packages/tequila/simulators/simulator_api.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Jordan Wigner Transformation

In [2]:
# Define the geometry
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

# Get molecule
mol_JW = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="JordanWigner").use_native_orbitals()

In [3]:
print('HF ', mol_JW.compute_energy('HF'))
fci = mol_JW.compute_energy('FCI')
print('FCI ', fci)

/home/liamj/Molecular Simulation/chemical-graph/tequila-beta/lib/python3.12/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)


HF  -1.8291374124430182
FCI  -1.9961503255188089


### Method 1: G1 SPA

In [4]:
U_JW_SPA = mol_JW.make_ansatz("SPA", edges=[(0,1),(2,3)])
H_JW_G1 = mol_JW.make_hamiltonian()

# Unitary for two orbital rotations (parameterized by 'a' and 'b')
U0 = mol_JW.UR(0,1,'a') + mol_JW.UR(2,3,'b')
U_JW_G1 = U0 + U_JW_SPA + U0.dagger()

res = tq.minimize(tq.ExpectationValue(H=H_JW_G1, U=U_JW_G1), silent=True)

U_JW_G1_mapped = U_JW_G1.map_variables(variables=res.variables)

for gate in U_JW_G1_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G1 = tq.compile(U_JW_G1_mapped, backend="cirq")

energy_JW_G1 = res.energy
print(f"difference from fci: {abs(res.energy-fci)*1000} mH")

print(f"Original number of measurements: {len(H_JW_G1.keys())}")
reduced_measurements_JW_G1 = reduce_measurements_naive(H_JW_G1, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G1)}")

No variables given...
difference from fci: 39.89570428503231 mH
Original number of measurements: 361
Reduced number of measurements: 103


### Method 2: G1 SPA with optimized orbitals

In [5]:
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol_JW,circuit=U_JW_SPA, initial_guess=guess, silent=True).molecule
H_JW_G1_optimized_orbitals = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1_optimized_orbitals,U=U_JW_SPA), silent=True)

U_JW_G1_optimized_orbitals_fixed_params = U_JW_SPA.map_variables(variables=res.variables)
cirq_JW_G1_optimized_orbitals = tq.compile(U_JW_G1_optimized_orbitals_fixed_params, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
energy_JW_G1_optimized_orbitals = res.energy

print(f"Original number of measurements: {len(H_JW_G1_optimized_orbitals.keys())}")
reduced_measurements_JW_G1_optimized_orbitals = reduce_measurements_naive(H_JW_G1_optimized_orbitals, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G1_optimized_orbitals)}")

No variables given...
difference from fci: 16.26244097546259 meh?

Original number of measurements: 361
Reduced number of measurements: 103


### Method 3: G2

In [6]:
# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol_JW.UR(1,2,'c')
UC = mol_JW.UC(1,2,'d')
U_JW_G2 = U_JW_SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1,U=U_JW_G2), silent=True)

U_JW_G2_mapped = U_JW_G2.map_variables(variables=res.variables)

for gate in U_JW_G2_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G2 = tq.compile(U_JW_G2_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
energy_JW_G2 = res.energy

print(f"Original number of measurements: {len(H_JW_G1.keys())}")
reduced_measurements_JW_G2 = reduce_measurements_naive(H_JW_G1, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G2)}")

No variables given...
difference from fci: 16.69940439785944 meh?

Original number of measurements: 361
Reduced number of measurements: 103


### Method 4: G2 with optimized Orbitals

In [7]:
U_JW_G2_optimized_orbitals = U_JW_SPA
U_JW_G2_optimized_orbitals += mol_JW.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol_JW.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_JW_G2_optimized_orbitals += mol_JW.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol_JW.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_JW_G2_optimized_orbitals += mol_JW.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol_JW.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1_optimized_orbitals,U=U_JW_G2_optimized_orbitals),silent=True)

U_JW_G2_optimized_orbitals_mapped = U_JW_G2_optimized_orbitals.map_variables(variables=res.variables)

for gate in U_JW_G2_optimized_orbitals_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G2_optimized_orbitals = tq.compile(U_JW_G2_optimized_orbitals_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
energy_JW_G2_optimized_orbitals = res.energy

print(f"Original number of measurements: {len(H_JW_G1_optimized_orbitals.keys())}")
reduced_measurements_JW_G2_optimized_orbitals = reduce_measurements_naive(H_JW_G1_optimized_orbitals, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G2_optimized_orbitals)}")

No variables given...
difference from fci: 9.697128646922515 meh?
Original number of measurements: 361
Reduced number of measurements: 103


# Parity Tapering Transformation

In [8]:
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="TaperedBinary").use_native_orbitals()

In [9]:
# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)

HF  -1.8291374124430178
FCI  -1.9961503255188089


### Method 1: G1

In [10]:
U_PT_SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H_PT_G1 = mol.make_hamiltonian()

# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U_PT_G1 = U0 + U_PT_SPA + U0.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1,U=U_PT_G1),silent=True)

U_PT_G1_mapped = U_PT_G1.map_variables(variables=res.variables)

for gate in U_PT_G1_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G1 = tq.compile(U_PT_G1_mapped, backend="cirq")

energy_PT_G1 = res.energy

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")

print(f"Original number of measurements: {len(H_PT_G1.keys())}")
reduced_measurements_PT_G1 = reduce_measurements_naive(H_PT_G1, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G1)}")

No variables given...
difference from fci: 39.89570428502609 meh?
Original number of measurements: 317
Reduced number of measurements: 88


### Method 2: G1 with optimized Orbitals

In [11]:
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=U_PT_SPA, initial_guess=guess, silent=True).molecule
H_PT_G1_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1_opt,U=U_PT_SPA), silent=True)

U_PT_G1_optimized_orbitals_fixed_params = U_PT_SPA.map_variables(variables=res.variables)
cirq_PT_G1_optimized_orbitals = tq.compile(U_PT_G1_optimized_orbitals_fixed_params, backend="cirq")

energy_PT_G1_optimized_orbitals = res.energy

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1_opt.keys())}")
reduced_measurements_PT_G1_optimized_orbitals = reduce_measurements_naive(H_PT_G1_opt, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G1_optimized_orbitals)}")

No variables given...
difference from fci: 16.26244097546281 meh?

Original number of measurements: 325
Reduced number of measurements: 84


### Method 3: G2

In [12]:
# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_PT_G2 = U_PT_SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1,U=U_PT_G2), silent=True)

U_PT_G2_mapped = U_PT_G2.map_variables(variables=res.variables)

for gate in U_PT_G2_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G2 = tq.compile(U_PT_G2_mapped, backend="cirq")

energy_PT_G2 = res.energy

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1.keys())}")
reduced_measurements_PT_G2 = reduce_measurements_naive(H_PT_G1, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G2)}")

No variables given...
difference from fci: 16.699404397387816 meh?

Original number of measurements: 317
Reduced number of measurements: 88


### Method 4: G2 with Optimized Orbitals

In [13]:
# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_PT_G2_optimized_orbitals = U_PT_SPA
U_PT_G2_optimized_orbitals += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_PT_G2_optimized_orbitals += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_PT_G2_optimized_orbitals += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1_opt,U=U_PT_G2_optimized_orbitals),silent=True)

U_PT_G2_optimized_orbitals_mapped = U_PT_G2_optimized_orbitals.map_variables(variables=res.variables)

for gate in U_PT_G2_optimized_orbitals_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G2_optimized_orbitals = tq.compile(U_PT_G2_optimized_orbitals_mapped, backend="cirq")

energy_PT_G2_optimized_orbitals = res.energy

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1_opt.keys())}")
reduced_measurements_PT_G2_optimized_orbitals = reduce_measurements_naive(H_PT_G1_opt, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G2_optimized_orbitals)}")

No variables given...
difference from fci: 9.697128646920739 meh?

Original number of measurements: 325
Reduced number of measurements: 84


# Get all circuits as Gates Lab Suite circuits (for taking data)

In [14]:

# Parameterized circuits
gates_JW_G1 = cirq_to_gates(cirq_JW_G1.circuit, 8, "Jordan-Wigner G1")
gates_JW_G1_optimized_orbitals = cirq_to_gates(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")
gates_JW_G2 = cirq_to_gates(cirq_JW_G2.circuit, 8, "Jordan-Wigner G2")
gates_JW_G2_optimized_orbitals = cirq_to_gates(cirq_JW_G2_optimized_orbitals.circuit, 8, "Jordan-Wigner G2 w/ optimized orbitals")

gates_PT_G1 = cirq_to_gates(cirq_PT_G1.circuit, 8, "Parity Tapering G1")
gates_PT_G1_optimized_orbitals = cirq_to_gates(cirq_PT_G1_optimized_orbitals.circuit, 8, "Parity Tapering G1 w/ optimized orbitals")
gates_PT_G2 = cirq_to_gates(cirq_PT_G2.circuit, 8, "Parity Tapering G2")
gates_PT_G2_optimized_orbitals = cirq_to_gates(cirq_PT_G2_optimized_orbitals.circuit, 8, "Parity Tapering G2 w/ optimized orbitals")


# Circuits with fixed, optimal parameters
gates_JW_G1_optimized_orbitals_fixed_params = cirq_to_gates(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")

# Get all circuits as Qibo circuits (for simulations)

In [15]:

# Parameterized circuits
qibo_JW_G1 = cirq_to_qibo(cirq_JW_G1.circuit, 8, "Jordan-Wigner G1")
qibo_JW_G1_optimized_orbitals = cirq_to_qibo(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")
qibo_JW_G2 = cirq_to_qibo(cirq_JW_G2.circuit, 8, "Jordan-Wigner G2")
qibo_JW_G2_optimized_orbitals = cirq_to_qibo(cirq_JW_G2_optimized_orbitals.circuit, 8, "Jordan-Wigner G2 w/ optimized orbitals")

qibo_PT_G1 = cirq_to_qibo(cirq_PT_G1.circuit, 8, "Parity Tapering G1")
qibo_PT_G1_optimized_orbitals = cirq_to_qibo(cirq_PT_G1_optimized_orbitals.circuit, 8, "Parity Tapering G1 w/ optimized orbitals")
qibo_PT_G2 = cirq_to_qibo(cirq_PT_G2.circuit, 8, "Parity Tapering G2")
qibo_PT_G2_optimized_orbitals = cirq_to_qibo(cirq_PT_G2_optimized_orbitals.circuit, 8, "Parity Tapering G2 w/ optimized orbitals")

# Circuits with fixed, optimal parameters
qibo_JW_G1_optimized_orbitals_fixed_params = cirq_to_qibo(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")


#### Check that cirq circuit gets correct energy

In [24]:
# Get the final state vector
simulator = cirq.Simulator()

result_JW_G1 = simulator.simulate(cirq_JW_G1.circuit)
result_JW_G1_optimized_orbitals = simulator.simulate(cirq_JW_G1_optimized_orbitals.circuit)
result_JW_G2 = simulator.simulate(cirq_JW_G2.circuit)
result_JW_G2_optimized_orbitals = simulator.simulate(cirq_JW_G2_optimized_orbitals.circuit)

result_PT_G1 = simulator.simulate(cirq_PT_G1.circuit)
result_PT_G1_optimized_orbitals = simulator.simulate(cirq_PT_G1_optimized_orbitals.circuit)
result_PT_G2 = simulator.simulate(cirq_PT_G2.circuit)
result_PT_G2_optimized_orbitals = simulator.simulate(cirq_PT_G2_optimized_orbitals.circuit)

state_vector_JW_G1 = result_JW_G1.final_state_vector
state_vector_JW_G1_optimized_orbitals = result_JW_G1_optimized_orbitals.final_state_vector
state_vector_JW_G2 = result_JW_G2.final_state_vector
state_vector_JW_G2_optimized_orbitals = result_JW_G2_optimized_orbitals.final_state_vector

state_vector_PT_G1 = result_PT_G1.final_state_vector
state_vector_PT_G1_optimized_orbitals = result_PT_G1_optimized_orbitals.final_state_vector
state_vector_PT_G2 = result_PT_G2.final_state_vector
state_vector_PT_G2_optimized_orbitals = result_PT_G2_optimized_orbitals.final_state_vector

qubits = cirq.LineQubit.range(8)

cirq_H_JW_G1, _ = convert_tq_H_to_cirq_H(H_JW_G1, qubits)
cirq_H_JW_G1_optimized_orbitals, _ = convert_tq_H_to_cirq_H(H_JW_G1_optimized_orbitals, qubits)

cirq_H_PT_G1, _ = convert_tq_H_to_cirq_H(H_PT_G1, qubits)
cirq_H_PT_G1_optimized_orbitals, _ = convert_tq_H_to_cirq_H(H_PT_G1_opt, qubits)

qubit_map = {}
for i in range(len(qubits)):
    qubit_map[qubits[i]] = i

# Calculate the expectation value
energy_JW_G1_cirq = cirq_H_JW_G1.expectation_from_state_vector(state_vector_JW_G1, qubit_map=qubit_map)
energy_JW_G1_optimized_orbitals_cirq = cirq_H_JW_G1_optimized_orbitals.expectation_from_state_vector(state_vector_JW_G1_optimized_orbitals, qubit_map=qubit_map)
energy_JW_G2_cirq = cirq_H_JW_G1.expectation_from_state_vector(state_vector_JW_G2, qubit_map=qubit_map)
energy_JW_G2_optimized_orbitals_cirq = cirq_H_JW_G1_optimized_orbitals.expectation_from_state_vector(state_vector_JW_G2_optimized_orbitals, qubit_map=qubit_map)

energy_PT_G1_cirq = cirq_H_PT_G1.expectation_from_state_vector(state_vector_PT_G1, qubit_map=qubit_map)
energy_PT_G1_optimized_orbitals_cirq = cirq_H_PT_G1_optimized_orbitals.expectation_from_state_vector(state_vector_PT_G1_optimized_orbitals, qubit_map=qubit_map)
energy_PT_G2_cirq = cirq_H_PT_G1.expectation_from_state_vector(state_vector_PT_G2, qubit_map=qubit_map)
energy_PT_G2_optimized_orbitals_cirq = cirq_H_PT_G1_optimized_orbitals.expectation_from_state_vector(state_vector_PT_G2_optimized_orbitals, qubit_map=qubit_map)


print("tq energy : cirq/tq EV energy")
print(f"{energy_JW_G1.real} : {energy_JW_G1_cirq.real}")
print(f"{energy_JW_G1_optimized_orbitals.real} : {energy_JW_G1_optimized_orbitals_cirq.real}")
print(f"{energy_JW_G2.real} : {energy_JW_G2_cirq.real}")
print(f"{energy_JW_G2_optimized_orbitals.real} : {energy_JW_G2_optimized_orbitals_cirq.real}")

print(f"{energy_PT_G1.real} : {energy_PT_G1_cirq.real}")
print(f"{energy_PT_G1_optimized_orbitals.real} : {energy_PT_G1_optimized_orbitals_cirq.real}")
print(f"{energy_PT_G2.real} : {energy_PT_G2_cirq.real}")
print(f"{energy_PT_G2_optimized_orbitals.real} : {energy_PT_G2_optimized_orbitals_cirq.real}")

tq energy : cirq/tq EV energy
-1.9562546212337766 : -1.956254594167206
-1.9798878845433463 : -1.9798879202636583
-1.9794509211209494 : -1.9794509370935032
-1.9864531968718864 : -1.9864533176619044
-1.9562546212337828 : -1.9562546977985926
-1.979887884543346 : -1.979887920263657
-1.979450921121421 : -1.97945133994675
-1.9864531968718881 : -1.9864536794241197


### Convert H to a dictionary of Pauli strings

In [25]:
H_JW_G1_dict = convert_tq_H_to_dict_H(H_JW_G1, 8)
H_JW_G1_optimized_orbitals_dict = convert_tq_H_to_dict_H(H_JW_G1_optimized_orbitals, 8)
H_JW_G2_dict = convert_tq_H_to_dict_H(H_JW_G1, 8)
H_JW_G2_optimized_orbitals_dict = convert_tq_H_to_dict_H(H_JW_G1_optimized_orbitals, 8)

H_PT_G1_dict = convert_tq_H_to_dict_H(H_PT_G1, 6)
H_PT_G1_optimized_orbitals_dict = convert_tq_H_to_dict_H(H_PT_G1_opt, 6)
H_PT_G2_dict = convert_tq_H_to_dict_H(H_PT_G1, 6)
H_PT_G2_optimized_orbitals_dict = convert_tq_H_to_dict_H(H_PT_G1_opt, 6)

### Simulate state probabilities using Cirq
Then calculate expectation values, then energy

In [27]:
# JW Transformation, G1
res_dict_JW_G1 = simulate_data_cirq(H_JW_G1_dict, cirq_JW_G1.circuit, 8)
evs_JW_G1 = convert_population_data_to_expectation_values(res_dict_JW_G1)
print(get_energy_from_expectation_values(H_JW_G1_dict, evs_JW_G1))

# JW Transformation, G1 with optimal hamiltonian
res_dict_JW_G1_optimized_orbitals = simulate_data_cirq(H_JW_G1_optimized_orbitals_dict, cirq_JW_G1_optimized_orbitals.circuit, 8)
evs_JW_G1_optimized_orbitals = convert_population_data_to_expectation_values(res_dict_JW_G1_optimized_orbitals)
print(get_energy_from_expectation_values(H_JW_G1_optimized_orbitals_dict, evs_JW_G1_optimized_orbitals))

# JW Transformation, G2
res_dict_JW_G2 = simulate_data_cirq(H_JW_G2_dict, cirq_JW_G2.circuit, 8)
evs_JW_G2 = convert_population_data_to_expectation_values(res_dict_JW_G2)
print(get_energy_from_expectation_values(H_JW_G1_dict, evs_JW_G2))

# JW Transformation, G2 with optimal hamiltonian
res_dict_JW_G2_optimized_orbitals = simulate_data_cirq(H_JW_G1_dict, cirq_JW_G1.circuit, 8)
evs_JW_G2_optimized_orbitals = convert_population_data_to_expectation_values(res_dict_JW_G2_optimized_orbitals)
print(get_energy_from_expectation_values(H_JW_G1_optimized_orbitals_dict, evs_JW_G2_optimized_orbitals))


# Parity Transformation, G1
res_dict_PT_G1 = simulate_data_cirq(H_PT_G1_dict, cirq_PT_G1.circuit, 6)
evs_PT_G1 = convert_population_data_to_expectation_values(res_dict_PT_G1)
print(get_energy_from_expectation_values(H_PT_G1_dict, evs_PT_G1))

# Parity Transformation, G1 with optimal hamiltonian
res_dict_PT_G1_optimized_orbitals = simulate_data_cirq(H_PT_G1_optimized_orbitals_dict, cirq_PT_G1_optimized_orbitals.circuit, 6)
evs_PT_G1_optimized_orbitals = convert_population_data_to_expectation_values(res_dict_PT_G1_optimized_orbitals)
print(get_energy_from_expectation_values(H_PT_G1_optimized_orbitals_dict, evs_PT_G1_optimized_orbitals))

# Parity Transformation, G2
res_dict_PT_G2 = simulate_data_cirq(H_PT_G2_dict, cirq_PT_G2.circuit, 6)
evs_PT_G2 = convert_population_data_to_expectation_values(res_dict_PT_G2)
print(get_energy_from_expectation_values(H_PT_G1_dict, evs_PT_G2))

# Parity Transformation, G2 with optimal hamiltonian
res_dict_PT_G2_optimized_orbitals = simulate_data_cirq(H_PT_G2_optimized_orbitals_dict, cirq_PT_G2_optimized_orbitals.circuit, 6)
evs_PT_G2_optimized_orbitals = convert_population_data_to_expectation_values(res_dict_PT_G2_optimized_orbitals)
print(get_energy_from_expectation_values(H_PT_G1_optimized_orbitals_dict, evs_PT_G2_optimized_orbitals))

-1.9649524886216567
-1.7017386361692293
-1.9876876568223216
-1.314640279931664
-1.7588093268734863
-1.8408134716463134
-1.7786648441144242
-1.8347226308235796
